<a href="https://colab.research.google.com/github/Mainabryan/loan-approval-prediction-logistic-regressio/blob/main/Loan_Approval_predictor(logistic_regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import the neccesary libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score



## Load the datset

In [4]:
data = pd.read_csv('/content/loan_data.csv')

## EDA (EXPALTORY DATA ANALYSIS)

In [5]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LN0001,Male,Yes,1,Graduate,No,5849,0.0,146.0,360.0,1.0,Urban,1.0
1,LN0002,Female,No,0,Graduate,Yes,4583,0.0,128.0,360.0,1.0,Semiurban,1.0
2,LN0003,Male,Yes,3+,Graduate,No,6000,2250.0,265.0,360.0,1.0,Rural,1.0
3,LN0004,Male,No,0,Not Graduate,No,2583,2358.0,120.0,360.0,0.0,Urban,0.0
4,LN0005,Female,Yes,2,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Semiurban,1.0


In [6]:
data.tail()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
496,LN0496,Male,Yes,2,Not Graduate,No,3333,0.0,120.0,360.0,0.0,Semiurban,0.0
497,LN0497,Female,No,0,Graduate,Yes,2500,0.0,100.0,360.0,1.0,Urban,1.0
498,LN0498,Male,Yes,1,Graduate,No,6250,0.0,210.0,360.0,1.0,Rural,1.0
499,LN0499,Female,Yes,0,Not Graduate,No,2083,0.0,80.0,360.0,0.0,Semiurban,0.0
500,LN0500,Male,No,0,Graduate,No,7667,0.0,200.0,360.0,1.0,Urban,1.0


This is the first step where you bring your .csv file into your working environment

## EDA (EXPALTORY DATA ANALYSIS)

# MIssing values

In [7]:
data.isnull().sum()

,0
Loan_ID,0
Gender,0
Married,0
Dependents,0
Education,0
Self_Employed,0
ApplicantIncome,0
CoapplicantIncome,2
LoanAmount,2
Loan_Amount_Term,2


In [10]:
# Fill numerical columns with median
data['CoapplicantIncome'] = data['CoapplicantIncome'].fillna(data['CoapplicantIncome'].median())
data['LoanAmount'] = data['LoanAmount'].fillna(data['LoanAmount'].median())

# Fill categorical / discrete numeric columns with mode
data['Loan_Amount_Term'] = data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mode()[0])
data['Credit_History'] = data['Credit_History'].fillna(data['Credit_History'].mode()[0])
data['Property_Area'] = data['Property_Area'].fillna(data['Property_Area'].mode()[0])